In [43]:
import numpy as np
import numpy.random as nprand
import matplotlib.pyplot as plt
import pandas as pd
import math as math

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif 
from sklearn.model_selection import cross_val_score

Data Loading and preprocessing:

In [44]:
# The original data = df
df = pd.read_csv('data.csv',
    header=None,
    index_col=False).T

Select labels, light and feature

In [45]:
lesions=df.iloc[:,1].apply(pd.to_numeric)
light=df.iloc[:,2].apply(pd.to_numeric)
features=df.iloc[:,3:-1].apply(pd.to_numeric)


In [51]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = lesions.values.reshape(len(lesions), 1)
lesions_encoded = onehot_encoder.fit_transform(integer_encoded)

In [94]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(features, lesions_encoded, test_size=0.5,stratify=lesions)


In [100]:
import keras.layers as layers
from keras.models import Model
import tensorflow as tf


def getModel(inputSize):
    inputs = layers.Input(shape=inputSize)
    x=layers.Dense(256)(inputs)
    x_expand=tf.expand_dims(x, axis=2)
    cnn_o1 = layers.Conv1D(
        filters=32,
        kernel_size=16,
        # Use 'same' padding so outputs have the same shape as inputs.
        padding='same')(x_expand)

    query_seq_encoding=layers.Dense(64)(inputs)
    # query_seq_encoding=tf.expand_dims(query_seq_encoding, axis=2)
    query_value_attention_seq = layers.Attention()(
        [cnn_o1, cnn_o1])

    query_value_attention = layers.GlobalAveragePooling1D()(
        query_value_attention_seq)

    concatenated = layers.Concatenate()(
        [x, query_value_attention])
    x=layers.Dense(32)(concatenated)
    x=layers.Dense(16)(x)
    output=layers.Dense(3,activation='softmax')(x)

    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer='nadam',loss='categorical_crossentropy',metrics=['accuracy'])
    model.summary()
    return model




In [ ]:
model=getModel(X_train.shape[1])
history=model.fit(X_train,y_train,epochs=500,batch_size=64,validation_split=0.2)

In [101]:
from sklearn.model_selection import LeaveOneOut
import keras.layers as layers
from keras.models import Model
import tensorflow as tf

loo = LeaveOneOut()
models=[]
features=np.array(features)
lesions_encoded=np.array(lesions_encoded)

y_true, y_output = list(), list()
for train_index, test_index in loo.split(features):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = features[train_index,:], features[test_index,:]
    y_train, y_test = lesions_encoded[train_index,:], lesions_encoded[test_index,:]
    model=getModel(X_train.shape[1])
    history=model.fit(X_train,y_train,epochs=200,batch_size=32,validation_split=0.3)
    yhat = model.predict(X_test)
    y_true.append(y_test[0])
    y_output.append(yhat[0])
y_test=y_true

TRAIN: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151] TEST: [0]
Model: "model_33"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_44 (InputLayer)           [(None, 697)]        0                                            
______________________________________________________

KeyboardInterrupt: 

In [96]:
from sklearn.metrics import classification_report

labelNames=['hyperplasic', 'serrated', 'adenoma']
y_output=model.predict(X_test)
y_output_transformed= onehot_encoder.inverse_transform(y_output)
y_test_transformed= onehot_encoder.inverse_transform(y_test)
print(classification_report(y_test_transformed, y_output_transformed, labels=[1,2,3]))



              precision    recall  f1-score   support

           1       0.86      0.57      0.69        21
           2       0.25      0.87      0.39        15
           3       0.70      0.17      0.28        40

    accuracy                           0.42        76
   macro avg       0.60      0.54      0.45        76
weighted avg       0.65      0.42      0.41        76



0.42105263157894735